In [5]:
import pandas as pd
import numpy as np

In [6]:
file_path = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [7]:
df.head ()

,Emotions,Text
0,joy,My first feeling of falling in love. Coming a...
1,joy,When I saw that the mark I had obtained in an ...
2,joy,When I learnt that I had been admitted to the ...
3,anger,In a hurry to get to lectures and the car woul...
4,guilt,"Not attending school, when am not sick - but ..."


In [8]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

1150


In [9]:
df['Emotions'].value_counts()


Emotions
shame      182
fear       168
anger      166
disgust    166
sadness    161
guilt      155
joy        152
Name: count, dtype: int64

In [10]:
print (df['Emotions'])
print (df['Emotions'].shape)
print (df['Emotions'].values)

0         joy
1         joy
2         joy
3       anger
4       guilt
        ...  
1145    anger
1146    anger
1147    shame
1148      joy
1149      joy
Name: Emotions, Length: 1150, dtype: object
(1150,)
['joy' 'joy' 'joy' ... 'shame' 'joy' 'joy']


In [11]:
emotion_labels = list (set (df['Emotions'].values))

In [14]:


def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string


#extract dict
text = ''.join(df['Text'].astype(str))
tokenized_text = tokenize(text) #all tokens, including repeating
#print (tokenized_text)
vocab = set(tokenized_text)
#print(vocab)

tokenized_text = []
for index, sentence in enumerate(df['Text']):
    emotion_label = df['Emotions'][index]
    sentence = tokenize(sentence)
    tokenized_text.append (sentence)
    print(emotion_label, sentence)
print (tokenized_text)


joy ['my', 'first', 'feeling', 'of', 'falling', 'in', 'love', '.', 'coming', 'across', 'a', 'person', 'who', 'made', 'me', 'feel', 'love', ',', 'happiness', ',', 'joy', ',', 'giving', 'me', 'a', 'new', 'and', 'beautiful', 'future', ',', 'who', 'made', 'me', 'dream', '.']
joy ['when', 'i', 'saw', 'that', 'the', 'mark', 'i', 'had', 'obtained', 'in', 'an', 'exam', 'was', 'a', 'good', 'one', '.']
joy ['when', 'i', 'learnt', 'that', 'i', 'had', 'been', 'admitted', 'to', 'the', 'university', '.']
anger ['in', 'a', 'hurry', 'to', 'get', 'to', 'lectures', 'and', 'the', 'car', 'would', 'not', 'start', '.']
guilt ['not', 'attending', 'school', ',', 'when', 'am', 'not', 'sick', '-', 'but', 'in', 'need', 'of', 'mental', 'rest', '.', 'staff', 'on', 'your', 'return', 'query', 'your', '\\', 'sickness', '\\', '"', '-', 'it', 'is', 'easier', 'á']
fear ['when', 'i', 'went', 'up', 'the', 'stairs', 'in', 'a', 'high', ',', 'dark', 'house', 'full', 'of', 'shadows', ',', 'passed', 'a', 'door', 'and', 'had', 

In [15]:
from math import log

collection = [token for instance in tokenized_text for token in instance]
#print (collection)

#tf-idf

def calculate_tfidf(token, document):
    tf = np.log(document.count(token) / len(document)) if token in document else 0 # 1+log(#oftimestokenindoc/total#termsindoc)
    idf = log (1 + (len(tokenized_text) / collection.count(token))) # log(#ofdocs/#oftimestokenappearsincollection)
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []
for sentence in tokenized_text:
        tfidf_scores = {token: calculate_tfidf(token, sentence) for token in set(sentence)}
        tfidf_scores_list.append (tfidf_scores)
print (tfidf_scores_list)



[{'my': -3.562999250443909, 'made': -9.09622847286316, 'in': -4.53859639554407, 'of': -4.45400839154573, 'love': -11.509169990334897, 'across': -21.159690364856843, 'person': -11.10079855366671, 'new': -17.272194699080124, 'first': -11.658835931487902, 'me': -3.4864228463669242, 'dream': -20.139962692754377, '.': -1.4574546915584876, 'giving': -21.159690364856843, 'feeling': -13.979008641420819, 'coming': -15.977054388210437, 'a': -2.368723142548951, 'joy': -16.564870442940386, 'beautiful': -18.159564465451048, 'happiness': -25.059466954877674, ',': -2.450372764401422, 'falling': -20.139962692754377, 'who': -7.691304506004488, 'and': -3.593644882895194, 'feel': -12.94192775857124, 'future': -19.34968925691931}, {'a': -2.344733404478971, 'that': -4.486437493957534, 'when': -3.106048422207394, 'had': -3.997440855261848, 'saw': -9.352082133866164, 'was': -2.934025295778005, 'the': -2.3217569128962556, 'mark': -14.471134605091658, 'an': -6.539288068729793, 'good': -9.612608770861378, 'i': 

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.sparse import csr_matrix
import numpy as np

# Convert the list of tokenized sentences into a list of strings
corpus = [' '.join(sentence) for sentence in tokenized_text]

# Create a CountVectorizer to convert the corpus into a document-term matrix
count_vectorizer = CountVectorizer(vocabulary=vocab)
X_count = count_vectorizer.fit_transform(corpus)

# Create a TfidfTransformer to compute the TF-IDF scores
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_count)


# Convert the TF-IDF matrix to CSR format if needed
if not isinstance(X_tfidf, csr_matrix):
    X_tfidf = X_tfidf.tocsr()

#print (corpus)
#print(X_tfidf)

In [29]:
tfidf_df = pd.DataFrame(X_tfidf)

tfidf_df.to_csv('tfidf_representations.csv', index=False)

tfidf_df.to_excel('tfidf_representations.xlsx', index=False)